In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')
import pandas as pd

In [3]:
from src.llm_finetuning.TrainingDataGenerationService import TrainingDataGenerationService

In [4]:
shortlisted_articles_df = pd.read_csv('attributes_1000_shortlisted_articles_Feb_19.csv')

In [5]:
TrainingDataGenerator = TrainingDataGenerationService(shortlisted_articles_df=shortlisted_articles_df, folder_path='/Users/username/Desktop/ML/Recommendations/arcane/notebooks/llms/prompt-refinemenet-18-feb-2023',model='gpt-4')

In [9]:
len(TrainingDataGenerator.shortlisted_articles_df)

1173

In [6]:
shortlisted_articles_df['all_content_tokens'] = shortlisted_articles_df['full_content'].apply(lambda x: len(TrainingDataGenerator.encoder.encode(x)))

In [10]:
TrainingDataGenerator.get_gpt4_attributes_for_article_ids(fraction=1)

completed 50 in 118.27990818023682 seconds
completed 100 in 248.94330596923828 seconds
completed 150 in 368.1834270954132 seconds
completed 200 in 482.02743005752563 seconds
completed 250 in 591.8838729858398 seconds
completed 300 in 706.3727412223816 seconds
completed 350 in 827.9119689464569 seconds
completed 400 in 921.6106412410736 seconds
completed 450 in 1037.3360950946808 seconds
completed 500 in 1151.0613429546356 seconds
completed 550 in 1264.3340148925781 seconds
completed 600 in 1368.896956205368 seconds
completed 650 in 1475.0333890914917 seconds
now 20 invalid responses
completed 700 in 1580.4480760097504 seconds
completed 750 in 1682.3313100337982 seconds
completed 800 in 1785.8359830379486 seconds
completed 850 in 1894.472097158432 seconds
completed 900 in 2006.1392121315002 seconds
completed 950 in 2122.244528055191 seconds
completed 1000 in 2223.6898329257965 seconds
completed 1050 in 2328.9982540607452 seconds
completed 1100 in 2430.1684880256653 seconds
done in 2536.

1144

In [11]:
len(TrainingDataGenerator.invalid_response_dict)

29

In [18]:
from concurrent.futures import as_completed, ThreadPoolExecutor

In [19]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(TrainingDataGenerator.fix_json_for_invalid_response, article_id) for article_id in list(TrainingDataGenerator.invalid_response_dict.keys())]
    for future in as_completed(futures):
        json_generated, parsed_response = future.result()
        if json_generated:
            TrainingDataGenerator.save_attributes(response_json=parsed_response)
            TrainingDataGenerator.invalid_response_dict.pop(parsed_response['article_id'])

json not properly generated for 65bb79cdf3aa2a77a58bfe97 at Feb 19, 2024 04:23
json not properly generated for 65316eb01e5cc42b1b13e628 at Feb 19, 2024 04:24
json not properly generated for 6560cea7e1d8342798a45325 at Feb 19, 2024 04:24
json not properly generated for 65c443c3da99411d4747695a at Feb 19, 2024 04:24


In [21]:
len(TrainingDataGenerator.invalid_response_dict)

4

In [7]:
TrainingDataGenerator.system_tokens

893

In [8]:
TrainingDataGenerator.content_tokens

1536

In [9]:
shortlisted_articles_df['corrected_tokens'] = shortlisted_articles_df['all_content_tokens'].apply(lambda x: min(x, 1536))

In [10]:
import pandas as pd

In [19]:
from copy import deepcopy

In [20]:
invalid_json = deepcopy(TrainingDataGenerator.invalid_response_dict['65316a681e5cc42b1b13dc29']['model_response'])

In [15]:
import json

In [7]:
b = {
  "summary": "!👄! Ad violations: According to ASCI, personal care product companies produced the most objectionable ads in FY22 and 1Q-3Q FY23. \n !📈! Rising trend: ASCI noted a 272% rise in the number of upheld objections from 2019 to 2023. The personal care industry contributes to 12% of all ads deemed objectionable. \n !📲! Influencers and D2C Brands: 68% of the ads processed in the personal care category were linked to influencers, while 84% of violative ads were from D2C brands.",
  "summary_critique": "The summary includes key details from the article such as the role of D2C brands and influencers in the rise of objectionable ads in the personal care sector. It adheres to the _summary_markdown_format_, is readable, accurately represents the information in the article, and is faithful to the content.",
  "improved_summary": "!👄! Personal Care Ad Violations: Personal care companies accounted for most 'objectionable' ads in FY22, and 1Q-3Q FY23, reveals an ASCI report. \n !📈! Growing Trend: There was a 272% surge in upheld objections from 2019 to 2023, the personal care sector making up 12% of all such ads. \n !📲! Influencer and D2C Impact: Influencers were linked to 68% of processed ads in the category, and D2C brands were associated with 84% of violative ads.",
  "top_queries": "ASCI report 2023; objectionable ads in personal care; rise in ad violations; role of influencers in ad violations; contribution of D2C brands to ad violations",
  "indian_or_international": "indian",
  "business_or_financial_article": True,
  "relevant_for_indians": True,
  "category": "business",
  "article_interest_duration_evaluation": "The information in this article is likely to be of interest to readers until the end of the financial year 2022-23, when new reports and data will be published.",
  "article_interest_duration": 14,
  "popularity_evaluation": 0.35,
  "popularity_evaluation_critique": "The popularity score is fair as the article highlights a concerning trend in a prevalent industry but lacks strong emotional appeal.",
  "final_reader_interest_score": 0.37,
  "final_headline_effectiveness_score": 0.35,
  "final_event_novelty_score": 0.4,
  "final_emotional_impact_score": 0.3,
  "improved_headline": "'Personal Care Brands Lead in Violation of Ad Codes: An ASCI Report'",
  "article_type": "fact",
  "article_sentiment": "NA"
}

In [8]:
list(b.keys())

['summary',
 'summary_critique',
 'improved_summary',
 'top_queries',
 'indian_or_international',
 'business_or_financial_article',
 'relevant_for_indians',
 'category',
 'article_interest_duration_evaluation',
 'article_interest_duration',
 'popularity_evaluation',
 'popularity_evaluation_critique',
 'final_reader_interest_score',
 'final_headline_effectiveness_score',
 'final_event_novelty_score',
 'final_emotional_impact_score',
 'improved_headline',
 'article_type',
 'article_sentiment']

In [17]:
fixed_json = json.dumps({
  "summary": "!👄! Ad violations: According to ASCI, personal care product companies produced the most objectionable ads in FY22 and 1Q-3Q FY23. \n !📈! Rising trend: ASCI noted a 272% rise in the number of upheld objections from 2019 to 2023. The personal care industry contributes to 12% of all ads deemed objectionable. \n !📲! Influencers and D2C Brands: 68% of the ads processed in the personal care category were linked to influencers, while 84% of violative ads were from D2C brands.",
  "summary_critique": "The summary includes key details from the article such as the role of D2C brands and influencers in the rise of objectionable ads in the personal care sector. It adheres to the _summary_markdown_format_, is readable, accurately represents the information in the article, and is faithful to the content.",
  "improved_summary": "!👄! Personal Care Ad Violations: Personal care companies accounted for most 'objectionable' ads in FY22, and 1Q-3Q FY23, reveals an ASCI report. \n !📈! Growing Trend: There was a 272% surge in upheld objections from 2019 to 2023, the personal care sector making up 12% of all such ads. \n !📲! Influencer and D2C Impact: Influencers were linked to 68% of processed ads in the category, and D2C brands were associated with 84% of violative ads.",
  "top_queries": "ASCI report 2023; objectionable ads in personal care; rise in ad violations; role of influencers in ad violations; contribution of D2C brands to ad violations",
  "indian_or_international": "indian",
  "business_or_financial_article": True,
  "relevant_for_indians": True,
  "category": "business",
  "article_interest_duration_evaluation": "The information in this article is likely to be of interest to readers until the end of the financial year 2022-23, when new reports and data will be published.",
  "article_interest_duration": 14,
  "popularity_evaluation": 0.35,
  "popularity_evaluation_critique": "The popularity score is fair as the article highlights a concerning trend in a prevalent industry but lacks strong emotional appeal.",
  "final_reader_interest_score": 0.37,
  "final_headline_effectiveness_score": 0.35,
  "final_event_novelty_score": 0.4,
  "final_emotional_impact_score": 0.3,
  "improved_headline": "'Personal Care Brands Lead in Violation of Ad Codes: An ASCI Report'",
  "article_type": "fact",
  "article_sentiment": "NA"
})

In [21]:
TrainingDataGenerator.invalid_response_dict['65316a681e5cc42b1b13dc29']['model_response'] = fixed_json

In [23]:
TrainingDataGenerator.invalid_response_dict['65316a681e5cc42b1b13dc29']['proper_json_generated'] = True

In [25]:
TrainingDataGenerator.save_attributes(response_json=TrainingDataGenerator.invalid_response_dict['65316a681e5cc42b1b13dc29'])

In [11]:
import json

In [17]:
json.loads(TrainingDataGenerator.valid_response_dict['651dd837a662d76276b7e69b']

{'summary': '!💡! Key Points: Rule-based investing strategy could help investors improve their financial outcomes by eliminating emotional biases; It involves defining the investment objective, identifying core investment competencies, selecting the technical and fundamental indicators, determining the Entry and exit rules, setting risk management and position sizing rules, and back-testing the strategy.',
 'summary_critique': 'The mentioned summary accurately corresponds to the article details primarily focusing on rule-based investment strategy. However, it lacks some crucial information such as using paper trading for testing the strategy and direct implementation after validating the strategy.',
 'improved_summary': '!💹! Investment Strategy: Following a rule-based investment strategy can curtail losses and improve trading performance. \\n !📝! Process: The strategy involves defining an investment objective, identifying core investment competencies, choosing relevant technical and fun

In [21]:
json.loads(TrainingDataGenerator.valid_response_dict['65316cb11e5cc42b1b13e0f4']['model_response'])

{'summary': "CCI Intervention: The Competition Commission of India (CCI) orders Denmark’s A.P. Moller-Maersk and Dubai’s DP World to withdraw customer advisories, which allegedly hamper growth at Mumbai's Jawaharlal Nehru Port Trust 🚢\nAntitrust Allegations: Probe ordered due to suspected antitrust violations. Singapore’s PSA International Pte Ltd alleges Maersk and DP World are hindering the growth of PSA’s terminal by colluding on port charges 🔍\nInterim Solution: The advisory withdrawal order is an interim measure. DP World and Maersk deny allegations, arguing the advisories were based on 'commercial justifications' 🛡️",
 'summary_critique': 'The summary accurately reflects the contents of the article, covering all important points. The adherence to the format is great, with labels and emoji making the summary engaging and easier to read. The grammar and punctuation are also correct, signifying good readability.',
 'improved_summary': "CCI Directive 🗒️: Denmark’s A.P. Moller-Maersk 

In [35]:
import json

In [43]:
k = {"summary": "<Summarize the article, focusing on: a. Thoroughness: Include essential details and expand on headline points. b. Readability: Ensure proper grammar, 2-3 points, up to 80 words. c. Faithfulness: Don't mention any details which are not part of the article. d. Accuracy: Verify numbers, dates, and events. f. Structure: Single-line points preceded by relevant label and emoji. g. Format: Use !emoji! label: point format without preamble/postamble, adhering to a 3-point maximum. We will call this format as _summary_markdown_format_ -> '!emoji1! label1: point1 \n !emoji2! label2: point2. The platform is visited by users of all age groups and hence do not use any inappropriate content or emojis>", "summary_critique": "<Evaluate the summary against the article for thoroughness, readability, faithfulness, accuracy, and the _summary_markdown_format_, noting any missed details or format issues>", "improved_summary": "<Improve the summary using feedback from the critique, following the _summary_markdown_format_>", "top_queries": "<List 5 relevant keywords/queries, separated by semicolons, without quotes to ensure JSON compatibility. keywords are short 1-2 words, while queries last from 3-6 words>", "indian_or_international": "<Specify if the article is \'indian\' or \'international\'>", "business_or_financial_article": "<True or False, based on the article\'s relevance to Indian corporations, investors, and policies impacting them>", "relevant_for_indians": "<True or False, considering the article\'s applicability to Indian readers. Most international news are not relevant for Indians, except if they are of very popular entities (like google, openai, Microsoft) or have global impact like fed changes>", "category": "<categorize the article into one of the 5 distinct categories based on the primary focus of article. a. irrelevant: if business_or_financial_article is False  b. financial: information of markets, financial instruments or company financial reports. c. business: information of operational, strategic, leadership or other non financial news of companies. d. economic_policy: information of economic trends or policy and regulatory impacts on larger industry. e. personal_finance: provides guidance on individual financial management, such as investment for personal goals, savings, taxation, budget updates, insurance or other financial products, directly targeting individual consumers and passive investors>", "article_interest_duration_evaluation": "<Analyze for how many days the information in the article will of interest to users after it is published>", "article_interest_duration": "<Determine the article\'s interest duration from options: 1, 3, 7, 14, 30, -1 (timeless), based on the evaluation>", "popularity_evaluation": "<Assess the article\'s potential popularity considering reader interest, headline effectiveness, event novelty, and emotional impact, with scores from 0 to 1. Be conservative with scores over 0.4>", "popularity_evaluation_critique": "<Critique the popularity assessment for possible overestimations or underestimations>", "final_reader_interest_score": "<0 to 1 float>", "final_headline_effectiveness_score": "<0 to 1 float>", "final_event_novelty_score": "<0 to 1 float>", "final_emotional_impact_score": "<0 to 1 float>", "improved_headline": "<Craft an engaging headline that captures the article\'s essence without resorting to clickbait>", "article_type": "<Identify the article as fact, opinion, analysis, educational, or sponsored, based on its content and presentation>", "article_sentiment": "<Determine if the article\'s sentiment is bullish, bearish, or NA (neutral)>"}

In [44]:
k.keys()

dict_keys(['summary', 'summary_critique', 'improved_summary', 'top_queries', 'indian_or_international', 'business_or_financial_article', 'relevant_for_indians', 'category', 'article_interest_duration_evaluation', 'article_interest_duration', 'popularity_evaluation', 'popularity_evaluation_critique', 'final_reader_interest_score', 'final_headline_effectiveness_score', 'final_event_novelty_score', 'final_emotional_impact_score', 'improved_headline', 'article_type', 'article_sentiment'])

In [45]:
json.dumps(k)

'{"summary": "<Summarize the article, focusing on: a. Thoroughness: Include essential details and expand on headline points. b. Readability: Ensure proper grammar, 2-3 points, up to 80 words. c. Faithfulness: Don\'t mention any details which are not part of the article. d. Accuracy: Verify numbers, dates, and events. f. Structure: Single-line points preceded by relevant label and emoji. g. Format: Use !emoji! label: point format without preamble/postamble, adhering to a 3-point maximum. We will call this format as _summary_markdown_format_ -> \'!emoji1! label1: point1 \\n !emoji2! label2: point2. The platform is visited by users of all age groups and hence do not use any inappropriate content or emojis>", "summary_critique": "<Evaluate the summary against the article for thoroughness, readability, faithfulness, accuracy, and the _summary_markdown_format_, noting any missed details or format issues>", "improved_summary": "<Improve the summary using feedback from the critique, following 

In [14]:
def convert_to_dicts(s):
    # Split the string into parts based on "\n" to get each point
    points = s.split("\\n")
    result = []

    for point in points:
        # Split each point into emoji and the rest based on the first "!"
        emoji_split = point.split("!", 2)
        if len(emoji_split) < 3:
            continue  # Skip if the format does not match expected
        
        emoji, rest = emoji_split[1], emoji_split[2]
        # Split the rest into label and point based on the first ":"
        label_point_split = rest.split(":", 1)
        if len(label_point_split) < 2:
            continue  # Skip if the format does not match expected

        label, point = label_point_split[0].strip(), label_point_split[1].strip()
        # Append the result as a dict to the result list
        result.append({"emoji": emoji, "label": label, "point": point})

    return result

# Example usage
s = "!emoji1! label1: point1 \\n !emoji2! label2: point2"
converted = convert_to_dicts(s)
print(converted)

[{'emoji': 'emoji1', 'label': 'label1', 'point': 'point1'}, {'emoji': 'emoji2', 'label': 'label2', 'point': 'point2'}]


In [15]:
s1 = '!💹! Investment Strategy: Following a rule-based investment strategy can curtail losses and improve trading performance. \\n !📝! Process: The strategy involves defining an investment objective, identifying core investment competencies, choosing relevant technical and fundamental indicators, and carefully crafting entry and exit rules.\\n !🛠️! Validation: The strategy must be back-tested with historical data, evaluated with paper trading in a simulated environment, and then implemented in live markets.'

In [16]:
convert_to_dicts(s1)

[{'emoji': '💹',
  'label': 'Investment Strategy',
  'point': 'Following a rule-based investment strategy can curtail losses and improve trading performance.'},
 {'emoji': '📝',
  'label': 'Process',
  'point': 'The strategy involves defining an investment objective, identifying core investment competencies, choosing relevant technical and fundamental indicators, and carefully crafting entry and exit rules.'},
 {'emoji': '🛠️',
  'label': 'Validation',
  'point': 'The strategy must be back-tested with historical data, evaluated with paper trading in a simulated environment, and then implemented in live markets.'}]

In [ ]:
1609 * 

In [5]:
shortlisted_articles_df

,url,source_id,published_time,short_description,tags,title,authors,ai_generated_info,cleaned_text,article_id,source_name,cat,subcat,fullcat,proportion,scaled_proportion,prob,is_selected,full_content
0,https://economictimes.indiatimes.com/news/inte...,6512cdcad01a9c8e86263e05,2023-07-29T16:46:00+05:30,Sony has adjusted the release dates of several...,"['Sony film releases', 'Spider Verse', 'sony',...",Sony delays major film releases: From 'Spider-...,['The Feed'],{'summary': {'OpenHermes_WatermelonSapphireZip...,Sony Pictures has made significant changes to ...,651df121a662d76276b82825,economic-times,news,international,economic-times-news-international,0.000229,0.004977,0.000648,True,headline: Sony delays major film releases: Fro...
1,https://www.bworldonline.com/infographics/2023...,65559d0c96fb32eef21a562f,2023-07-04T16:02:07+00:00,The Philippines ranked 102nd and 80th out of 1...,NaN,Philippines ranks 102nd and 80th in freedom an...,['CEDadiantiTyClea'],{'summary': {'OpenHermes_WatermelonSapphireZip...,Home Infographics Philippines ranks 102nd and ...,6555c1164b13023f9348ce12,business-world,infographics,NaN,business-world-infographics-,0.001185,0.025799,0.024488,True,headline: Philippines ranks 102nd and 80th in ...
2,https://zerodha.com/varsity/chapter/maintainin...,65291b1e9a2fbc229e5f29c9,2020-04-29T07:53:50+0000,Maintaining discipline is vital for trading su...,NaN,251. Maintaining Discipline : Innerworth -...,[],{'summary': {'OpenHermes_WatermelonSapphireZip...,The winning trader is a disciplined trader. Di...,652ebbbc1e5cc42b1b139a05,zerodha-varsity,varsity,NaN,zerodha-varsity-varsity-,0.000686,0.014937,0.000935,True,headline: 251. Maintaining Discipline : In...
3,https://www.livemint.com/market/patanjali-ofs-...,650046bd005149c49201269f,2023-07-13T17:14:17+05:30,Patanjali Foods' non-retail open offer for 2.5...,"['Patanjali foods', 'OFS', 'equity shares offe...",Patanjali OFS: Non-retail portion fully subscr...,['Kaustubh Bhosale'],{'summary': {'OpenHermes_WatermelonSapphireZip...,The non-retail portion of the open offer (OFS...,65316f2b1e5cc42b1b13f1fa,live-mint,market,NaN,live-mint-market-,0.001210,0.026338,0.024126,True,headline: Patanjali OFS: Non-retail portion fu...
4,https://www.livemint.com/market/live-blog/ange...,650046bd005149c49201269f,2024-01-23T09:17:53+05:30,"Angel One stock price went down today, 23 Jan ...","['Angel One Share Price Live', 'Angel One Stoc...",Angel One share price Today Live Updates : Ang...,['Livemint'],{'summary': {'OpenHermes_ApricotTangerineScall...,"Angel One Share Price Today : On the last day,...",65af640c6391ac19baa87413,live-mint,market,live-blog,live-mint-market-live-blog,0.000181,0.003937,0.000101,True,headline: Angel One share price Today Live Upd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,https://zerodha.com/varsity/chapter/surviving-...,65291b1e9a2fbc229e5f29c9,2020-12-17T06:02:28+0000,It may be hard not to feel a little disappoint...,NaN,Surviving a Slump : Innerworth - Mind over mar...,[],{'summary': {'OpenHermes_WatermelonSapphireZip...,"Despite what we might wish for, it’s hard to m...",652ebbd21e5cc42b1b139ae3,zerodha-varsity,varsity,NaN,zerodha-varsity-varsity-,0.000686,0.014937,0.279419,False,headline: Surviving a Slump : Innerworth - Min...
2308,https://zerodha.com/varsity/chapter/clear-and-...,65291b1e9a2fbc229e5f29c9,2020-01-29T06:11:39+0000,Goals direct and motivate. Without clear and s...,NaN,52. Clear and Specific Goals : Innerworth ...,[],{'summary': {'OpenHermes_WatermelonSapphireZip...,Trading can be tedious at times. Day in and da...,652ebb9f1e5cc42b1b13994c,zerodha-varsity,varsity,NaN,zerodha-varsity-varsity-,0.000686,0.014937,0.595580,False,headline: 52. Clear and Specific Goals : I...
2309,https://zerodha.com/varsity/chapter/valuation-...,65291b1e9a2fbc229e5f29c9,2022-05-12T04:30:44+0000,14.1 – Valuations basics We are at the last st...,NaN,14. Valuation (Part 1) – Overview : Integr...,[],{'summary': {'OpenH